In [1]:
import asyncio

import aiohttp

import aiomoex
import pandas as pd

In [87]:
async def main():
    # request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/boards/TQBR/securities.json"
    request_url = "https://iss.moex.com/iss/engines/stock/" "markets/shares/securities/SBER/trades.json"
    # arguments = {"securities.columns": ("SECID," "REGNUMBER," "LOTSIZE," "SHORTNAME")}
    # arguments = {"securities.columns": ("SECID", "REGNUMBER", "LOTSIZE", "SHORTNAME")}
# https://iss.moex.com/iss/engines/stock/markets/shares/securities/AFLT/trades.xml

    async with aiohttp.ClientSession() as session:
        iss = aiomoex.ISSClient(session, request_url, {'reversed': 1, 'limit': 1})
        data = await iss.get()
        df = pd.DataFrame(data["trades"])
        # df.set_index("SECID", inplace=True)
        # print(df.head(), "\n")
        # print(df.tail(), "\n")
        # df.info()

    return df

df_ticks = await main()

In [88]:
df_ticks

,TRADENO,TRADETIME,BOARDID,SECID,PRICE,QUANTITY,VALUE,PERIOD,TRADETIME_GRP,SYSTIME,BUYSELL,DECIMALS,TRADINGSESSION,TRADEDATE,TRADE_SESSION_DATE
0,12631604584,21:47:13,TQBR,SBER,319.29,35,111751.5,N,2147,2025-03-11 21:47:13,B,2,2,2025-03-11,2025-03-11


In [9]:
async def main():
    async with aiohttp.ClientSession() as session:
        data = await aiomoex.get_board_history(session, 'SNGSP')
        df = pd.DataFrame(data)
        df.set_index('TRADEDATE', inplace=True)
        print(df.head(), '\n')
        print(df.tail(), '\n')
        df.info()

await main()

           BOARDID  CLOSE    VOLUME         VALUE
TRADEDATE                                        
2014-06-09    TQBR  27.48  12674200  3.484352e+08
2014-06-10    TQBR  27.55  14035900  3.856417e+08
2014-06-11    TQBR  28.15  27208800  7.602146e+08
2014-06-16    TQBR  28.27  68059900  1.913160e+09
2014-06-17    TQBR  28.20  22101600  6.292844e+08 

           BOARDID   CLOSE    VOLUME         VALUE
TRADEDATE                                         
2025-03-04    TQBR  55.995  57512650  3.232298e+09
2025-03-05    TQBR  55.135  51382300  2.857517e+09
2025-03-06    TQBR  54.355  34269940  1.872701e+09
2025-03-07    TQBR  54.965  38897460  2.132026e+09
2025-03-10    TQBR  54.760  18958100  1.042672e+09 

<class 'pandas.core.frame.DataFrame'>
Index: 2717 entries, 2014-06-09 to 2025-03-10
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BOARDID  2717 non-null   object 
 1   CLOSE    2699 non-null   float64
 2   VOLUME   2717 no

In [15]:
from datetime import datetime, timedelta

In [48]:
im = {'D': 24, 'W': 7, 'H': 60, 'M': 31}

im.get('W', 24)

7

In [49]:
async def fetch_moex_data(ticker, timeframe='D', days=1825):
    """
    Асинхронная загрузка свечных данных с Московской биржи через aiomoex.
    """
    # Определяем конечную и начальную даты
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days)

    # Приводим к строковому формату YYYY-MM-DD
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    # Маппинг интервалов
    # interval_map = {'D': 24, 'W': 7, 'H': 60, 'M': 31}
    # interval = interval_map.get(timeframe, 24)  # По умолчанию дневные свечи
    interval = 1

    print(f"📊 Загружаем данные для {ticker} с {start_str} по {end_str} (таймфрейм: {timeframe})")

    async with aiohttp.ClientSession() as session:
        # Запрашиваем свечные данные
        data = await aiomoex.get_market_candles(
            session,
            security=ticker,
            interval=interval,
            start=start_str,
            end=end_str
        )

    # Проверяем, есть ли данные
    if not data:
        raise ValueError(f"❌ Не удалось получить данные для {ticker} за последние {days} дней")

    # Создаем DataFrame
    df = pd.DataFrame(data)

    # Переименовываем колонки
    column_mapping = {
        'begin': 'Date',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'value': 'Volume' # если volume, то объём торгов в штуках
    }
    df = df.rename(columns=column_mapping)

    # Преобразуем дату в datetime и ставим индекс
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Оставляем только нужные столбцы
    df = df[['Open', 'High', 'Low', 'Close', 'Volume']]

    print(f"✅ Загружено {len(df)} свечей для {ticker}")

    return df, start_str, end_str 

await main()

           BOARDID  CLOSE    VOLUME         VALUE
TRADEDATE                                        
2014-06-09    TQBR  27.48  12674200  3.484352e+08
2014-06-10    TQBR  27.55  14035900  3.856417e+08
2014-06-11    TQBR  28.15  27208800  7.602146e+08
2014-06-16    TQBR  28.27  68059900  1.913160e+09
2014-06-17    TQBR  28.20  22101600  6.292844e+08 

           BOARDID   CLOSE    VOLUME         VALUE
TRADEDATE                                         
2025-03-04    TQBR  55.995  57512650  3.232298e+09
2025-03-05    TQBR  55.135  51382300  2.857517e+09
2025-03-06    TQBR  54.355  34269940  1.872701e+09
2025-03-07    TQBR  54.965  38897460  2.132026e+09
2025-03-10    TQBR  54.760  18958100  1.042672e+09 

<class 'pandas.core.frame.DataFrame'>
Index: 2717 entries, 2014-06-09 to 2025-03-10
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BOARDID  2717 non-null   object 
 1   CLOSE    2699 non-null   float64
 2   VOLUME   2717 no

In [50]:
df, start_str, end_str = await fetch_moex_data("SBER", timeframe='m5', days=100)

📊 Загружаем данные для SBER с 2024-12-01 по 2025-03-11 (таймфрейм: m5)
✅ Загружено 62205 свечей для SBER


In [51]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2024-12-02 09:59:00,237.02,237.02,237.02,237.02,21824801.6
2024-12-02 10:00:00,237.09,237.49,237.06,237.15,145894993.9
2024-12-02 10:01:00,237.14,237.90,237.03,237.90,132170307.0
2024-12-02 10:02:00,237.89,238.10,237.44,237.71,162964868.8
2024-12-02 10:03:00,237.71,237.83,237.54,237.60,88916069.6
...,...,...,...,...,...
2025-03-11 20:14:00,320.99,321.43,320.98,321.32,66049697.2
2025-03-11 20:15:00,321.28,321.29,321.14,321.14,2708202.2
2025-03-11 20:16:00,321.14,321.19,320.91,321.12,20165899.8


In [31]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-03-12,185.15,187.87,173.14,175.91,3.564667e+10
2020-03-13,175.00,199.52,174.30,198.60,5.043423e+10
2020-03-16,194.00,196.70,186.00,195.85,3.346209e+10
2020-03-17,195.00,204.78,183.80,184.01,3.513444e+10
2020-03-18,179.00,182.20,172.15,174.27,2.743838e+10
...,...,...,...,...,...
2025-03-05,316.40,318.30,313.00,313.56,1.825876e+10
2025-03-06,313.60,317.43,313.57,315.18,1.007412e+10
2025-03-07,315.19,321.87,310.69,316.92,3.513136e+10


In [33]:
async def fetch_moex_ticks(ticker, board="TQBR"):
    """
    Асинхронная загрузка тиковых данных с Московской биржи через aiomoex.
    
    :param ticker: Тикер бумаги (например, "SBER")
    :param board: Торговая площадка (по умолчанию "TQBR" - основной рынок акций)
    :return: DataFrame с тиковыми данными
    """
    
    print(f"📊 Загружаем тиковые данные для {ticker}...")

    async with aiohttp.ClientSession() as session:
        # Запрос тиковых данных с Московской биржи
        data = await aiomoex.get_market_trades(
            session,
            security=ticker,
            board=board
        )

    # Проверяем, есть ли данные
    if not data:
        raise ValueError(f"❌ Не удалось получить тиковые данные для {ticker}")

    # Создаем DataFrame
    df = pd.DataFrame(data)

    # Переименовываем столбцы
    column_mapping = {
        'systime': 'Timestamp',  # Время сделки
        'price': 'Price',        # Цена сделки
        'quantity': 'Volume',    # Количество лотов
        'value': 'Value',        # Стоимость сделки (рубли)
        'seqnum': 'TradeID'      # ID сделки
    }
    df = df.rename(columns=column_mapping)

    # Преобразуем время в datetime
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])

    # Сортируем по времени (на всякий случай)
    df = df.sort_values(by="Timestamp")

    print(f"✅ Загружено {len(df)} тиков для {ticker}")

    return df

In [79]:
df_ticks = await main()

# Просмотр первых строк
df_ticks.head()

       TRADENO TRADETIME BOARDID SECID   PRICE  QUANTITY    VALUE PERIOD  \
0  12625901918  06:50:00    SMAL  SBER  323.54         1   323.54      N   
1  12625901926  06:51:48    SMAL  SBER  318.47         3   955.41      N   
2  12625901927  06:51:48    SMAL  SBER  316.01         5  1580.05      N   
3  12625901930  06:53:55    SMAL  SBER  318.43         3   955.29      N   
4  12625901931  06:53:55    SMAL  SBER  318.44         6  1910.64      N   

   TRADETIME_GRP              SYSTIME BUYSELL  DECIMALS TRADINGSESSION  \
0            650  2025-03-11 06:50:09       B         2              0   
1            651  2025-03-11 06:51:49       B         2              0   
2            651  2025-03-11 06:51:49       S         2              0   
3            653  2025-03-11 06:53:55       B         2              0   
4            653  2025-03-11 06:53:55       B         2              0   

    TRADEDATE TRADE_SESSION_DATE  
0  2025-03-11         2025-03-11  
1  2025-03-11         2025-0

,TRADENO,TRADETIME,BOARDID,SECID,PRICE,QUANTITY,VALUE,PERIOD,TRADETIME_GRP,SYSTIME,BUYSELL,DECIMALS,TRADINGSESSION,TRADEDATE,TRADE_SESSION_DATE
0,12625901918,06:50:00,SMAL,SBER,323.54,1,323.54,N,650,2025-03-11 06:50:09,B,2,0,2025-03-11,2025-03-11
1,12625901926,06:51:48,SMAL,SBER,318.47,3,955.41,N,651,2025-03-11 06:51:49,B,2,0,2025-03-11,2025-03-11
2,12625901927,06:51:48,SMAL,SBER,316.01,5,1580.05,N,651,2025-03-11 06:51:49,S,2,0,2025-03-11,2025-03-11
3,12625901930,06:53:55,SMAL,SBER,318.43,3,955.29,N,653,2025-03-11 06:53:55,B,2,0,2025-03-11,2025-03-11
4,12625901931,06:53:55,SMAL,SBER,318.44,6,1910.64,N,653,2025-03-11 06:53:55,B,2,0,2025-03-11,2025-03-11


In [80]:
df_ticks

,TRADENO,TRADETIME,BOARDID,SECID,PRICE,QUANTITY,VALUE,PERIOD,TRADETIME_GRP,SYSTIME,BUYSELL,DECIMALS,TRADINGSESSION,TRADEDATE,TRADE_SESSION_DATE
0,12625901918,06:50:00,SMAL,SBER,323.54,1,323.54,N,650,2025-03-11 06:50:09,B,2,0,2025-03-11,2025-03-11
1,12625901926,06:51:48,SMAL,SBER,318.47,3,955.41,N,651,2025-03-11 06:51:49,B,2,0,2025-03-11,2025-03-11
2,12625901927,06:51:48,SMAL,SBER,316.01,5,1580.05,N,651,2025-03-11 06:51:49,S,2,0,2025-03-11,2025-03-11
3,12625901930,06:53:55,SMAL,SBER,318.43,3,955.29,N,653,2025-03-11 06:53:55,B,2,0,2025-03-11,2025-03-11
4,12625901931,06:53:55,SMAL,SBER,318.44,6,1910.64,N,653,2025-03-11 06:53:55,B,2,0,2025-03-11,2025-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,12626011829,09:26:54,TQBR,SBER,317.34,50,158670.00,N,926,2025-03-11 09:26:55,B,2,0,2025-03-11,2025-03-11
4996,12626012018,09:27:03,TQBR,SBER,317.34,5,15867.00,N,927,2025-03-11 09:27:03,B,2,0,2025-03-11,2025-03-11
4997,12626012030,09:27:05,TQBR,SBER,317.33,2,6346.60,N,927,2025-03-11 09:27:05,S,2,0,2025-03-11,2025-03-11
4998,12626012031,09:27:05,TQBR,SBER,317.34,45,142803.00,N,927,2025-03-11 09:27:05,B,2,0,2025-03-11,2025-03-11


In [77]:
df_ticks[df_ticks['TRADETIME'] == '11:00:03']

,TRADENO,TRADETIME,BOARDID,SECID,PRICE,QUANTITY,VALUE,PERIOD,TRADETIME_GRP,SYSTIME,BUYSELL,DECIMALS,TRADINGSESSION,TRADEDATE,TRADE_SESSION_DATE
